In [ ]:
# UltimateSynopticArchive VERSION 1.00
# Released: 11/10/2022

# *IMPORTANT* steps to using this code successfully:
#    1: Be sure to enter *ALL* required information in code cell #3 below
#    ---> This includes the save location... You may need to create a new folder where this notebook is located
#    2: *AFTER* you have entered all of the correct information, run *ALL* cells
#    3: Wait at the last code cell for prompts on successful (or not) map creation
#    4: After prompted that the map(s) have been created, check your save folder to make sure they are there

# Troubleshooting checklist:
#    -The code ran successfully, but my maps have no or incorrect data plotted on them
#        -Try re-running the code
#        -If the issue persists, try running a different case date (I recommend 12/28/2021 as I know it works)
#            -If changing the case date solves the issue, your case date's data may be corrupted
#    -I am getting an error during "URL Fetching"
#        -Is your case date between January 1, 1979 and a month ago? If not, data is not availible for your case
#        -If yes, then make sure your date is correctly input into code cell #3 (ie: leading zero for month/day 1-9)
#        -If the issue persists, try running a different case date (I recommend 12/28/2021 as I know it works)
#            -If changing the case date solves the issue, your case date's data may be missing
#    -I am getting an error thrown on the "plt.savefig" line
#        -Make sure your save location in code cell #3 exists (ie: you have created a folder and inserted the name in cell #3)

# Known Issues:
#    -Missing data at the Prime Meridian (Europe projection *ONLY*)
#        -Some solutions to this probelm exist online, but are not immediately compatible with PyGrib Grids
#    -Missing state boundaries on the Northern Hemisphere projection
#        -Need to find a solution to have cartopy plot only a certain country's state/providence boundaries
#    -925mb frontogenesis may struggle to plot for CFS cases (pre-2007) ---> Unknown cause at this time

# Future Plans:
#    -Add PWAT map for moisture transport identification (especially winter cases)
#    -Add 850mb winds map for LLJ identification
#    -Add precipitation type, MSLP, and 1000-500mb thickness map
#    -Add handling for multiple timestep input
#    -Add support for saving the 
#    -Add Southern Hemisphere Views (Need to handle flipping wind barbs, switching vorticity sign)
#        -Might be easier to have a completely different notebook for Southern Hemisphere cases

# If you: 
#    -Find any bugs while using the code (not already outlined in "known issues" above)
#    -Are having problems with the code (and have went through the troubleshooting checklist above) 
#    -Have suggestions for new features/map types to be added to the code
#    ---> Please let me know! Twitter is probably the best way to contact me: @KniprathWx

In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import numpy as np
import pygrib
from metpy.units import units
import metpy.calc as mpcalc
import cmocean as cmo
import scipy.ndimage as ndimage
import urllib.request 
from colorama import Fore
import matplotlib.colors as colors

In [ ]:
#Choose the year/month/day/hour of your case
month = '12' #Leading zero **REQUIRED** for months 1-9
day = '28' #Leading zero **REQUIRED** for days 1-9
year = '2021' #1979-Present (data not availible for up to the most recent month)
hour = '00' #00, 06, 12, or 18 

#Selct your sector view (ie: what projection you want) ---> *Only Northern Hemisphere Projections Are Currently Availible
SECTOR = 'CONUS' #Options are 'CONUS', 'North America', 'Europe', or 'Asia'. (Europe has missing data at the prime meridian)

#Choose which synoptic maps you would like created - enter 'Y' for yes or 'N' for no
jet250 = 'Y' #250mb winds and geopotential heights
vorticity500 = 'N' #500mb cyclonic vorticity, geopotential heights, and winds
moisture850 = 'N' #850mb dew points >8°C, geopotential heights, temperatures, and winds
frontogenesis925 = 'N' #925mb frontogenesis, geopotential heights, and winds
surface = 'N' #2m temp, freezing line, MSLP, and 1000-500mb thickness

#Choose if you want wind speed in mph or knots (this applies to ALL maps)
WIND = 'mph' #Enter 'mph' or 'kt'

#Choose if ypu want your surface map 2m tempertures in °C of °F
TEMP = 'F' #Enter 'F' or 'C' 

#Choose the folder (within the folder you are currently in) that you want the maps to save to
saveLoc = "SaveLoc"

In [ ]:
#URL Fetching

#Quick string creation for URL fetching
date = year+month
date2 = year+month+day
date3 = year+month+day+hour

try:
    #Set the URL
    url = 'https://www.ncei.noaa.gov/data/global-forecast-system/access/grid-004-0.5-degree/analysis/'+date+'/'+date2+'/gfs_4_'+date2+'_'+hour+'00_000.grb2'

    #Set file name
    out_Name = 'GFS_'+date2+'_'+hour+'z'

    #Download the file
    urllib.request.urlretrieve(url, out_Name)
    
    #Set the model name for file name creation
    modelName = 'GFS'
except:
    #Set the URL
    url = 'https://www.ncei.noaa.gov/oa/prod-cfs-reanalysis/6-hourly-by-pressure-level/'+year+'/'+date+'/'+date2+'/pgbh01.gdas.'+date3+'.grb2'
    
    #Set file name
    out_Name = 'CFS_'+date2+'_'+hour+'z'

    #Download the file
    urllib.request.urlretrieve(url, out_Name)
    
    #Set the model name for file name creation
    modelName = 'CFS'

In [ ]:
#This opens the grb file and displays all varibles present in the file
grbs = pygrib.open(out_Name)
grbs.seek(0)

for grb in grbs:
    print(grb)

In [ ]:
#Find Time & Date Variables in grb File 
T = grb.validDate

T = str(T)    

time = T[11:13]

time = "%sz" % (time)

year = T[0:4]

month = T[5:7]

day = T[8:10]

#Create strings for file names
date = "%s/%s/%s" % (month, day, year)
saveDate = "%s-%s-%s" % (month, day, year)

#Create text string for map title
validT = "Valid: %s %s" % (time, date)

#Create File Names For Later
mapNameJet250 = "250mbJet_"+modelName
fileNameJet250 = "%s/%s_%s_%s.png" % (saveLoc, mapNameJet250, saveDate, time)

mapNameVorticity500 = "500mbVorticity_"+modelName
fileNameVorticity500 = "%s/%s_%s_%s.png" % (saveLoc, mapNameVorticity500, saveDate, time)

mapNameMoisture850 = "850mbMoisture_"+modelName
fileNameMoisture850 = "%s/%s_%s_%s.png" % (saveLoc, mapNameMoisture850, saveDate, time)

mapNameFrontogenesis925 = "925mbFrontogenesis_"+modelName
fileNameFrontogenesis925 = "%s/%s_%s_%s.png" % (saveLoc, mapNameFrontogenesis925, saveDate, time)

mapNameSurface = "Surface_"+modelName
fileNameSurface = "%s/%s_%s_%s.png" % (saveLoc, mapNameSurface, saveDate, time)

In [ ]:
def JetMap250():
    #Reset the grb file
    grbs.seek(0)
    grb = grbs[1]

    #Data selection for 250mb Heights
    HGT_250mb = grbs.select(name='Geopotential Height', typeOfLevel='isobaricInhPa', level=250)[0]
    HGT_250mb = HGT_250mb.values
    HGT_250mb = units('meter') * HGT_250mb
    
    #Data selection for U Wind Component at 250mb
    UWND_250mb = grbs.select(name='U component of wind', typeOfLevel='isobaricInhPa', level=250)[0]
    UWND_250mb = UWND_250mb.values
    UWND_250mb = units('m/s') * UWND_250mb

    #Data selection for V Wind Component at 250mb
    VWND_250mb = grbs.select(name='V component of wind', typeOfLevel='isobaricInhPa', level=250)[0]
    VWND_250mb = VWND_250mb.values
    VWND_250mb = units('m/s') * VWND_250mb
    
    #Calculate wind speed at 250mb
    sped_250mb = mpcalc.wind_speed(UWND_250mb, VWND_250mb).to(WIND)
    
    #Convert units to decameters for geopotential heights
    HGT_250mb = HGT_250mb / 10

    #Apply slight smoothing to parameters
    sped_250mb = ndimage.gaussian_filter(sped_250mb, sigma=1, order=0)
    HGT_250mb = ndimage.gaussian_filter(HGT_250mb, sigma=1, order=0)
    
    #Take in lats and lons from grb file
    lats, lons = grb.latlons()

    #Creating the map figure
    map_crs = ccrs.LambertConformal(central_longitude=-100, central_latitude=35, standard_parallels=(30, 60))
    datacrs = ccrs.PlateCarree()
    fig = plt.figure(1, figsize=(14, 12))
    ax = plt.subplot(1, 1, 1, projection=datacrs )

    #Running a if-elif-else to find correct projection
    if SECTOR == 'CONUS':
        ax.set_extent([-127.5, -64.5, 20, 55], ccrs.PlateCarree())
        WINDSLICE = 6
    elif SECTOR == 'North America':
        ax.set_extent([-168, -30, 10, 74], ccrs.PlateCarree())
        WINDSLICE = 10
    elif SECTOR == 'Europe':
        ax.set_extent([-25, 47, 29, 72], ccrs.PlateCarree())
        WINDSLICE = 7
    elif SECTOR == 'Asia':
        ax.set_extent([27, 180, 5, 78.2], ccrs.PlateCarree())
        WINDSLICE = 12

    #Add features to the map
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'ocean', '10m', facecolor='steelblue', zorder=50))
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'land', '10m', facecolor='gainsboro', zorder=50))
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'lakes', '10m', facecolor='steelblue', zorder=60))
    if SECTOR == 'CONUS':
        ax.add_feature(cfeature.NaturalEarthFeature('cultural', 'admin_1_states_provinces_lakes', '10m', facecolor='none', edgecolor='black', linewidth=1, zorder=70))
    else:
        ax.add_feature(cfeature.NaturalEarthFeature('cultural', 'admin_0_countries', '10m', facecolor='none', edgecolor='black', linewidth=1, zorder=70))
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'coastline', '10m', facecolor='none', edgecolor='black', linewidth=0.5, zorder=70))

    #Setting up the ranges for contoured wind speeds
    clevs_sped_250mb = [50, 75, 100, 125, 150, 175, 200, 225, 250]
    
    #*OLD* Custom Colortable
    #Creating a custom colormap with matplotlib colors 
    #cmap_data = ['yellow', 'orange', 'orangered', 'saddlebrown', 'darkviolet', 'deeppink', 'violet', 'lavenderblush']
    #cmapWIND = colors.ListedColormap(cmap_data, 'temperature')
    #norm = colors.BoundaryNorm(clevs_sped_250mb, cmapWIND.N)
    
    #Small function to truncate the wind speed color map to make it more visulally appealing
    def truncate_colormap(cmap, minval=0.0, maxval=1.0, n=100):
        new_cmap = colors.LinearSegmentedColormap.from_list(
            'trunc({n},{a:.2f},{b:.2f})'.format(n=cmap.name, a=minval, b=maxval),
            cmap(np.linspace(minval, maxval, n)))
        return new_cmap

    #arr = np.linspace(0, 50, 100).reshape((10, 10))
    #fig, ax = plt.subplots(ncols=2)

    cmap = plt.get_cmap('gist_ncar')
    gist_ncar_REFORMED = truncate_colormap(cmap, 0.18, 1)
    #ax[0].imshow(arr, interpolation='nearest', cmap=cmap)  
    #ax[1].imshow(arr, interpolation='nearest', cmap=gist_ncar_REFORMED)
    #plt.show()

    #Plot colorfill for wind speeds at 250mb
    cf = ax.contourf(lons, lats, sped_250mb, clevs_sped_250mb, cmap=cmo.tools.lighten(gist_ncar_REFORMED, 0.6), transform=datacrs, zorder=65)

    #Plot a grey contour line at transitions values for wind speeds at 250mb
    cs2 = ax.contour(lons, lats, sped_250mb, clevs_sped_250mb, colors='grey', transform=datacrs, alpha=0.6, linewidths=1, zorder=66)

    #Add colorbar for shaded wind speeds at 250mb
    cb = plt.colorbar(cf, orientation='horizontal', pad=0, aspect=50)
    cb.set_label('Wind Speed ('+WIND+')')

    #Plot contours for geopotential heights at 250mb
    clevs_HGT_250mb = np.arange(0, 1200, 12)
    cs = ax.contour(lons, lats, HGT_250mb, clevs_HGT_250mb, colors='black', transform=datacrs, zorder=80)
    plt.clabel(cs, fmt='%d')

    #Plot wind barbs every sixth element
    wind_slice = (slice(None, None, WINDSLICE), slice(None, None, WINDSLICE))           
    ax.barbs(lons[wind_slice], lats[wind_slice], UWND_250mb.to(WIND)[wind_slice].m, VWND_250mb[wind_slice].to(WIND).m, pivot='middle', color='black', transform=datacrs, length=6.5, zorder=67)

    #Add titles
    plt.title('250mb '+modelName+' Geopotential Heights (black, dam), Wind Speed (shaded, '+WIND+'), and Wind Barbs ('+WIND+')', loc='left')
    props = dict(boxstyle='round', facecolor='white')
    ax.text(0.828, 0.9825, validT, transform=ax.transAxes, fontsize=11.5, verticalalignment='top', bbox=props, zorder=100)

    #Save figure to your computer (save loction outlined near date selection)
    plt.savefig(fileNameJet250, dpi=200)

    #Clear figure variables for next figure creation
    plt.clf()

In [ ]:
def VorticityMap500():
    #Reset the grb file
    grbs.seek(0)
    grb = grbs[1]

    #Data selection for 500mb Heights
    HGT_500mb = grbs.select(name='Geopotential Height', typeOfLevel='isobaricInhPa', level=500)[0]
    HGT_500mb = HGT_500mb.values
    HGT_500mb = units('meter') * HGT_500mb

    #Data selection for U Wind Component at 500mb
    UWND_500mb = grbs.select(name='U component of wind', typeOfLevel='isobaricInhPa', level=500)[0]
    UWND_500mb = UWND_500mb.values
    UWND_500mb = units('m/s') * UWND_500mb

    #Data selection for V Wind Component at 500mb
    VWND_500mb = grbs.select(name='V component of wind', typeOfLevel='isobaricInhPa', level=500)[0]
    VWND_500mb = VWND_500mb.values
    VWND_500mb = units('m/s') * VWND_500mb

    #Data selection for Vorticity at 500mb
    VORT_500mb = grbs.select(name='Absolute vorticity', typeOfLevel='isobaricInhPa', level=500)[0]
    VORT_500mb = VORT_500mb.values
    VORT_500mb = units('1/s') * VORT_500mb
    
    #Convert units to decameters for geopotential heights
    HGT_500mb = HGT_500mb / 10
    
    #Convert 500mb Vorticity into usable units
    VORT_500mb = VORT_500mb * (1 / 10 ** -5)

    #Apply slight smoothing to parameters
    VORT_500mb = ndimage.gaussian_filter(VORT_500mb, sigma=1, order=0)
    HGT_500mb = ndimage.gaussian_filter(HGT_500mb, sigma=1, order=0)
    
    #Small function to truncate the vorticity color map to make it more visulally appealing
    def truncate_colormap(cmap, minval=0.0, maxval=1.0, n=100):
        new_cmap = colors.LinearSegmentedColormap.from_list(
            'trunc({n},{a:.2f},{b:.2f})'.format(n=cmap.name, a=minval, b=maxval),
            cmap(np.linspace(minval, maxval, n)))
        return new_cmap

    #arr = np.linspace(0, 50, 100).reshape((10, 10))
    #fig, ax = plt.subplots(ncols=2)

    cmap = plt.get_cmap('hot_r')
    HOT_R = truncate_colormap(cmap, 0.10, 1)
    #ax[0].imshow(arr, interpolation='nearest', cmap=cmap)  
    #ax[1].imshow(arr, interpolation='nearest', cmap=new_cmap)
    #plt.show()
    
    #Take in lats and lons from grb file
    lats, lons = grb.latlons()

    #Creating the map figure
    map_crs = ccrs.LambertConformal(central_longitude=-100, central_latitude=35, standard_parallels=(30, 60))
    datacrs = ccrs.PlateCarree()
    fig = plt.figure(1, figsize=(14, 12))
    ax = plt.subplot(1, 1, 1, projection=datacrs )

    #Running a if-elif-else to find correct projection
    if SECTOR == 'CONUS':
        ax.set_extent([-127.5, -64.5, 20, 55], ccrs.PlateCarree())
        WINDSLICE = 6
    elif SECTOR == 'North America':
        ax.set_extent([-168, -30, 10, 74], ccrs.PlateCarree())
        WINDSLICE = 10
    elif SECTOR == 'Europe':
        ax.set_extent([-25, 47, 29, 72], ccrs.PlateCarree())
        WINDSLICE = 7
    elif SECTOR == 'Asia':
        ax.set_extent([27, 180, 5, 78.2], ccrs.PlateCarree())
        WINDSLICE = 12

    #Add features to the map
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'ocean', '10m', facecolor='steelblue', zorder=50))
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'land', '10m', facecolor='gainsboro', zorder=50))
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'lakes', '10m', facecolor='steelblue', zorder=60))
    if SECTOR == 'CONUS':
        ax.add_feature(cfeature.NaturalEarthFeature('cultural', 'admin_1_states_provinces_lakes', '10m', facecolor='none', edgecolor='black', linewidth=1, zorder=70))
    else:
        ax.add_feature(cfeature.NaturalEarthFeature('cultural', 'admin_0_countries', '10m', facecolor='none', edgecolor='black', linewidth=1, zorder=70))
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'coastline', '10m', facecolor='none', edgecolor='black', linewidth=0.5, zorder=70))

    #Plot colorfill of 500mb vorticity
    clevs_VORT = np.arange(10,51,1)
    cf = ax.contourf(lons, lats, VORT_500mb, clevs_VORT, cmap=HOT_R, transform=datacrs, zorder=65)

    #Add colorbar for shaded vorticity at 500mb
    cb = plt.colorbar(cf, orientation='horizontal', pad=0, aspect=50)
    cb.set_label('Cyclonic Vorticity (1/sec*10^-5)')

    #Plot contours for geopotential heights at 500mb
    clevs_HGT_500mb = np.arange(0, 800, 6)
    cs = ax.contour(lons, lats, HGT_500mb, clevs_HGT_500mb, colors='black', transform=datacrs, zorder=80)
    plt.clabel(cs, fmt='%d')

    #Plot wind barbs every sixth element
    wind_slice = (slice(None, None, WINDSLICE), slice(None, None, WINDSLICE)) 
    ax.barbs(lons[wind_slice], lats[wind_slice], UWND_500mb.to(WIND)[wind_slice].m, VWND_500mb[wind_slice].to(WIND).m, pivot='middle', color='black', transform=datacrs, length=6.5, zorder=67)

    #Add titles
    plt.title('500mb '+modelName+' Geopotential Heights (balck, dam), Cyclonic Vorticity (shaded, 1/sec*10^-5), and Wind Barbs ('+WIND+')', loc='left')
    props = dict(boxstyle='round', facecolor='white')
    ax.text(0.828, 0.9825, validT, transform=ax.transAxes, fontsize=11.5, verticalalignment='top', bbox=props, zorder=100)

    #Save figure to your computer (save loction outlined near date selection)
    plt.savefig(fileNameVorticity500, dpi=200)

    #Clear figure variables for next figure creation
    plt.clf()

In [ ]:
def MoistureMap850():
    #Reset the grb file
    grbs.seek(0)
    grb = grbs[1]

    #Data selection for 850mb Heights
    HGT_850mb = grbs.select(name='Geopotential Height', typeOfLevel='isobaricInhPa', level=850)[0]
    HGT_850mb = HGT_850mb.values
    HGT_850mb = units('meter') * HGT_850mb

    #Data selection for U Wind Component at 850mb
    UWND_850mb = grbs.select(name='U component of wind', typeOfLevel='isobaricInhPa', level=850)[0]
    UWND_850mb = UWND_850mb.values
    UWND_850mb = units('m/s') * UWND_850mb

    #Data selection for V Wind Component at 850mb
    VWND_850mb = grbs.select(name='V component of wind', typeOfLevel='isobaricInhPa', level=850)[0]
    VWND_850mb = VWND_850mb.values
    VWND_850mb = units('m/s') * VWND_850mb

    #Data selection for temperature at 850mb
    TMP_850mb = grbs.select(name='Temperature', typeOfLevel='isobaricInhPa', level=850)[0]
    TMP_850mb = TMP_850mb.values
    TMP_850mb = units('kelvin') * TMP_850mb
    TMP_850mbC = TMP_850mb.to('degC')
    
    #Data selection for RH at 850mb
    RH_850mb = grbs.select(name='Relative humidity', typeOfLevel='isobaricInhPa', level=850)[0]
    RH_850mb = RH_850mb.values
    RH_850mb = units('%') * RH_850mb
    
    #Calculate dew point at 850mb
    DEW_850mb = mpcalc.dewpoint_from_relative_humidity(TMP_850mb, RH_850mb).to('degC')

    #Apply slight smoothing to parameters
    TMP_850mbC = ndimage.gaussian_filter(TMP_850mbC, sigma=1, order=0)
    DEW_850mb = ndimage.gaussian_filter(DEW_850mb, sigma=1, order=0)
    HGT_850mb = ndimage.gaussian_filter(HGT_850mb, sigma=1, order=0)
    
    #Small function to truncate the dew point color map to make it more visulally appealing
    def truncate_colormap(cmap, minval=0.0, maxval=1.0, n=100):
        new_cmap = colors.LinearSegmentedColormap.from_list(
            'trunc({n},{a:.2f},{b:.2f})'.format(n=cmap.name, a=minval, b=maxval),
            cmap(np.linspace(minval, maxval, n)))
        return new_cmap

    #arr = np.linspace(0, 50, 100).reshape((10, 10))
    #fig, ax = plt.subplots(ncols=2)

    cmap = plt.get_cmap('terrain_r')
    TERRAIN_R = truncate_colormap(cmap, 0.48, 1)
    #ax[0].imshow(arr, interpolation='nearest', cmap=cmap)  
    #ax[1].imshow(arr, interpolation='nearest', cmap=TERRAIN_R)
    #plt.show()
    
    #Take in lats and lons from grb file
    lats, lons = grb.latlons()

    #Creating the map figure
    map_crs = ccrs.LambertConformal(central_longitude=-100, central_latitude=35, standard_parallels=(30, 60))
    datacrs = ccrs.PlateCarree()
    fig = plt.figure(1, figsize=(14, 12))
    ax = plt.subplot(1, 1, 1, projection=datacrs )

    #Running a if-elif-else to find correct projection
    if SECTOR == 'CONUS':
        ax.set_extent([-127.5, -64.5, 20, 55], ccrs.PlateCarree())
        WINDSLICE = 6
    elif SECTOR == 'North America':
        ax.set_extent([-168, -30, 10, 74], ccrs.PlateCarree())
        WINDSLICE = 10
    elif SECTOR == 'Europe':
        ax.set_extent([-25, 47, 29, 72], ccrs.PlateCarree())
        WINDSLICE = 7
    elif SECTOR == 'Asia':
        ax.set_extent([27, 180, 5, 78.2], ccrs.PlateCarree())
        WINDSLICE = 12

    #Add features to the map
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'ocean', '10m', facecolor='steelblue', zorder=50))
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'land', '10m', facecolor='gainsboro', zorder=50))
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'lakes', '10m', facecolor='steelblue', zorder=60))
    if SECTOR == 'CONUS':
        ax.add_feature(cfeature.NaturalEarthFeature('cultural', 'admin_1_states_provinces_lakes', '10m', facecolor='none', edgecolor='black', linewidth=1, zorder=70))
    else:
        ax.add_feature(cfeature.NaturalEarthFeature('cultural', 'admin_0_countries', '10m', facecolor='none', edgecolor='black', linewidth=1, zorder=70))
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'coastline', '10m', facecolor='none', edgecolor='black', linewidth=0.5, zorder=70))

    #Plot colorfill of 850mb dew point
    clevs_DEW = np.arange(8,23,2)
    cf = ax.contourf(lons, lats, DEW_850mb, clevs_DEW, cmap=cmo.tools.lighten(TERRAIN_R, 0.6), transform=datacrs, zorder=65)

    #Add colorbar for shaded dew point at 850mb
    cb = plt.colorbar(cf, orientation='horizontal', pad=0, aspect=50)
    cb.set_label('Dew Point >8°C')

    #Plot a grey contour line at transitions values for dew point at 850mb
    cs2 = ax.contour(lons, lats, DEW_850mb, clevs_DEW, colors='grey', transform=datacrs, alpha=0.6, linewidths=1, zorder=66)

    #Plot contours for geopotential heights at 850mb
    clevs_HGT_850mb = np.arange(0, 3000, 30)
    cs = ax.contour(lons, lats, HGT_850mb, clevs_HGT_850mb, colors='black', transform=datacrs, zorder=80)
    plt.clabel(cs, fmt='%d')

    #Plot 850mb temperatures below freezing
    clevs_tempcold = np.arange(-40, -1, 2)
    cs3 = ax.contour(lons, lats, TMP_850mbC, clevs_tempcold, colors='blue', linewidths=0.75, linestyles='dashed', transform=datacrs, zorder=75)
    plt.clabel(cs3, fmt='%d')

    #Plot 850mb temperatures above freezing
    clevs_tempwarm = np.arange(0, 56, 2)
    cs4 = ax.contour(lons, lats, TMP_850mbC, clevs_tempwarm, colors='red', linewidths=0.75, linestyles='dashed', transform=datacrs, zorder=75)
    plt.clabel(cs4, fmt='%d')

    #Plot wind barbs every sixth element
    wind_slice = (slice(None, None, WINDSLICE), slice(None, None, WINDSLICE)) 
    ax.barbs(lons[wind_slice], lats[wind_slice], UWND_850mb[wind_slice].to(WIND).m, VWND_850mb[wind_slice].to(WIND).m, pivot='middle', color='black', transform=datacrs, length=6.5, zorder=67)

    #Add titles
    plt.title('850mb '+modelName+' Geopotential Heights (balck, m), Dew Point (shaded, °C), Temperature (dashed, °C), and Wind Barbs ('+WIND+')', loc='left')
    props = dict(boxstyle='round', facecolor='white')
    ax.text(0.828, 0.9825, validT, transform=ax.transAxes, fontsize=11.5, verticalalignment='top', bbox=props, zorder=100)

    #Save figure to your computer (save loction outlined near date selection)
    plt.savefig(fileNameMoisture850, dpi=200)

    #Clear figure variables for next figure creation
    plt.clf()

In [ ]:
def FrontogenesisMap925():
    #Reset the grb file
    grbs.seek(0)
    grb = grbs[1]
    
    #Data selection for 925mb Heights
    HGT_925mb = grbs.select(name='Geopotential Height', typeOfLevel='isobaricInhPa', level=925)[0]
    HGT_925mb = HGT_925mb.values
    HGT_925mb = units('meter') * HGT_925mb
    
    #Data selection for 925mb Temperatures
    TEMP_925mb = grbs.select(name='Temperature', typeOfLevel='isobaricInhPa', level=925)[0]
    TEMP_925mb = TEMP_925mb.values
    TEMP_925mb = units('kelvin') * TEMP_925mb
    
    #Data selection for U Wind Component at 925mb
    UWND_925mb = grbs.select(name='U component of wind', typeOfLevel='isobaricInhPa', level=925)[0]
    UWND_925mb = UWND_925mb.values
    UWND_925mb = units('m/s') * UWND_925mb

    #Data selection for V Wind Component at 925mb
    VWND_925mb = grbs.select(name='V component of wind', typeOfLevel='isobaricInhPa', level=925)[0]
    VWND_925mb = VWND_925mb.values
    VWND_925mb = units('m/s') * VWND_925mb
    
    #Setting variables for calculating frontogenesis
    level925 = 925 * units.hPa
    lats, lons = grb.latlons()
    dx, dy = mpcalc.lat_lon_grid_deltas(lons, lats)

    #Calculating theta at 925mb for frontogenesis calculation
    THETA_925mb = mpcalc.potential_temperature(level925, TEMP_925mb)

    #Calculating frontogenesis
    FRONTO_925mb = mpcalc.frontogenesis(THETA_925mb, UWND_925mb, VWND_925mb, dx, dy)
    convert_to_per_100km_3h = 1000*100*3600*3
    FRONTO_925mb = FRONTO_925mb * convert_to_per_100km_3h
    
    #Apply slight smoothing to parameters
    #FRONTO_925mb = ndimage.gaussian_filter(FRONTO_925mb, sigma=1, order=0) * units('K/hour')
    HGT_925mb = ndimage.gaussian_filter(HGT_925mb, sigma=1, order=0) * units.meter
    
    #Convert heights from m to dam
    HGT_925mb = HGT_925mb / 10
    
    #Take in lats and lons from grb file
    lats, lons = grb.latlons()

    #Creating the map figure
    map_crs = ccrs.LambertConformal(central_longitude=-100, central_latitude=35, standard_parallels=(30, 60))
    datacrs = ccrs.PlateCarree()
    fig = plt.figure(1, figsize=(14, 12))
    ax = plt.subplot(1, 1, 1, projection=datacrs )

    #Running a if-elif-else to find correct projection
    if SECTOR == 'CONUS':
        ax.set_extent([-127.5, -64.5, 20, 55], ccrs.PlateCarree())
        WINDSLICE = 6
    elif SECTOR == 'North America':
        ax.set_extent([-168, -30, 10, 74], ccrs.PlateCarree())
        WINDSLICE = 10
    elif SECTOR == 'Europe':
        ax.set_extent([-25, 47, 29, 72], ccrs.PlateCarree())
        WINDSLICE = 7
    elif SECTOR == 'Asia':
        ax.set_extent([27, 180, 5, 78.2], ccrs.PlateCarree())
        WINDSLICE = 12

    #Add features to the map
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'ocean', '10m', facecolor='steelblue', zorder=50))
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'land', '10m', facecolor='gainsboro', zorder=50))
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'lakes', '10m', facecolor='steelblue', zorder=60))
    if SECTOR == 'CONUS':
        ax.add_feature(cfeature.NaturalEarthFeature('cultural', 'admin_1_states_provinces_lakes', '10m', facecolor='none', edgecolor='black', linewidth=1, zorder=70))
    else:
        ax.add_feature(cfeature.NaturalEarthFeature('cultural', 'admin_0_countries', '10m', facecolor='none', edgecolor='black', linewidth=1, zorder=70))
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'coastline', '10m', facecolor='none', edgecolor='black', linewidth=0.5, zorder=70))

    #Plot colorfill of 925mb Frontogenesis
    clevs_FRONTO = np.arange(0.5,17,0.5)
    cf = ax.contourf(lons, lats, FRONTO_925mb, clevs_FRONTO, cmap='cool', transform=datacrs, zorder=65)

    #Add colorbar for shaded Frontogenesis at 925mb
    cb = plt.colorbar(cf, orientation='horizontal', pad=0, aspect=50)
    cb.set_label('Frontogenesis (°C/100km/3hr)')

    #Plot contours for geopotential heights at 500mb
    clevs_HGT_925mb = np.arange(0, 300, 3)
    cs = ax.contour(lons, lats, HGT_925mb, clevs_HGT_925mb, colors='black', transform=datacrs, zorder=80)
    plt.clabel(cs, fmt='%d')

    #Plot wind barbs every sixth element
    wind_slice = (slice(None, None, WINDSLICE), slice(None, None, WINDSLICE)) 
    ax.barbs(lons[wind_slice], lats[wind_slice], UWND_925mb.to(WIND)[wind_slice].m, VWND_925mb[wind_slice].to(WIND).m, pivot='middle', color='black', transform=datacrs, length=6.5, zorder=67)

    #Add titles
    plt.title('925mb '+modelName+' Frontogenesis (shaded, °C/100km/3hr), Heights (black, dam), and Wind Barbs ('+WIND+')', loc='left')
    props = dict(boxstyle='round', facecolor='white')
    ax.text(0.828, 0.9825, validT, transform=ax.transAxes, fontsize=11.5, verticalalignment='top', bbox=props, zorder=100)

    #Save figure to your computer (save loction outlined near date selection)
    plt.savefig(fileNameFrontogenesis925, dpi=200)

    #Clear figure variables for next figure creation
    plt.clf()

In [ ]:
def SurfaceMap():
    #Reset the grb file
    grbs.seek(0)
    grb = grbs[1]
    
    #Data selection for 500mb Heights
    HGT_500mb = grbs.select(name='Geopotential Height', typeOfLevel='isobaricInhPa', level=500)[0]
    HGT_500mb = HGT_500mb.values
    HGT_500mb = units('meter') * HGT_500mb

    #Data selection for 1000mb Heights
    HGT_1000mb = grbs.select(name='Geopotential Height', typeOfLevel='isobaricInhPa', level=1000)[0]
    HGT_1000mb = HGT_1000mb.values
    HGT_1000mb = units('meter') * HGT_1000mb

    #Data selection for MSLP
    try: #This is handling for CFS vs GFS data as varible names differ
        MSLP = grbs.select(name='MSLP (Eta model reduction)')[0]
    except:
        MSLP = grbs.select(name='Mean sea level pressure')[0] 
    MSLP = MSLP.values
    MSLP = units('Pa') * MSLP
    MSLP = MSLP.to('hPa')

    #Data selection for MSLP
    SFC_temp = grbs.select(name='2 metre temperature')[0]
    SFC_temp = SFC_temp.values
    SFC_temp = units('K') * SFC_temp
    if TEMP == 'F':
        SFC_temp = SFC_temp.to('degF')
    else:
        SFC_temp = SFC_temp.to('degC')
    
    #Calculate 1000-500mb thickness
    Thickness = HGT_500mb - HGT_1000mb

    #Apply slight smoothing to parameters
    Thickness = ndimage.gaussian_filter(Thickness, sigma=1, order=0)
    MSLP = ndimage.gaussian_filter(MSLP, sigma=1, order=0)
    SFC_temp = ndimage.gaussian_filter(SFC_temp, sigma=1, order=0)
    
    #Take in lats and lons from grb file
    lats, lons = grb.latlons()

    #Creating the map figure
    map_crs = ccrs.LambertConformal(central_longitude=-100, central_latitude=35, standard_parallels=(30, 60))
    datacrs = ccrs.PlateCarree()
    fig = plt.figure(1, figsize=(14, 12))
    ax = plt.subplot(1, 1, 1, projection=datacrs )

    #Running a if-elif-else to find correct projection
    if SECTOR == 'CONUS':
        ax.set_extent([-127.5, -64.5, 20, 55], ccrs.PlateCarree())
        WINDSLICE = 6
    elif SECTOR == 'North America':
        ax.set_extent([-168, -30, 10, 74], ccrs.PlateCarree())
        WINDSLICE = 10
    elif SECTOR == 'Europe':
        ax.set_extent([-25, 47, 29, 72], ccrs.PlateCarree())
        WINDSLICE = 7
    elif SECTOR == 'Asia':
        ax.set_extent([27, 180, 5, 78.2], ccrs.PlateCarree())
        WINDSLICE = 12

    #Add features to the map
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'ocean', '10m', facecolor='steelblue', zorder=50))
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'land', '10m', facecolor='gainsboro', zorder=50))
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'lakes', '10m', facecolor='steelblue', zorder=60))
    if SECTOR == 'CONUS':
        ax.add_feature(cfeature.NaturalEarthFeature('cultural', 'admin_1_states_provinces_lakes', '10m', facecolor='none', edgecolor='black', linewidth=1, zorder=70))
    else:
        ax.add_feature(cfeature.NaturalEarthFeature('cultural', 'admin_0_countries', '10m', facecolor='none', edgecolor='black', linewidth=1, zorder=70))
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'coastline', '10m', facecolor='none', edgecolor='black', linewidth=0.5, zorder=70))

    #If-else statement for temperture plotting °F vs °C
    if TEMP == 'F':
        #Setting up the ranges for contoured temperatures
        clevs = np.arange(-40, 121, 5)

        #Creating a custom colortable using RGB values
        cmap_data = [(194/255, 190/255, 193/255),(235/255, 216/255, 230/255),(244/255,183/255,245/255),(237/255,137/255,239/255),(204/255,109/255,247/255),(213/255,0/255,255/255),(157/255,0/255,188/255),(106/255,0/255,156/255),(55/255,36/255,102/255),(44/255,81/255,216/255),(43/255,117/255,249/255),(44/255,158/255,252/255),(35/255,195/255,228/255),(24/255,219/255,197/255),(37/255,236/255,167/255),(74/255,248/255,128/255),(121/255,254/255,89/255),(164/255,252/255,60/255),(195/255,241/255,52/255),(223/255,223/255,55/255),(244/255,199/255,58/255),(253/255,172/255,52/255),(253/255,138/255,38/255),(243/255,99/255,21/255),(228/255,69/255,10/255),(206/255,45/255,4/255),(178/255,26/255,1/255),(142/255,10/255,1/255),(208/255,0/255,114/255),(255/255,0/255,142/255),(255/255,79/255,175/255),(255/255,125/255,196/255),(255/255,167/255,215/255)]
        cmap = colors.ListedColormap(cmap_data, 'temperature')
        norm = colors.BoundaryNorm(clevs, cmap.N)

        #Plotting 2m temperature with colorfill
        cf = ax.contourf(lons, lats, SFC_temp, np.arange(-40, 121, 5), cmap=cmo.tools.lighten(cmap, 0.6), transform=datacrs, zorder=65)

        #Creating a colorbar for 2m temperature
        cb = plt.colorbar(cf, orientation='horizontal', pad=0, aspect=50)
        cb.set_label('2m Temperature (°F)')

        #Plot a grey contour line at transition values for 2m temperatures
        cs22 = ax.contour(lons, lats, SFC_temp, np.arange(-40, 121, 5), linewidths=0.75, colors='grey', alpha=0.6, transform=datacrs, zorder=66)

        #Plot the freezing line
        cs28 = ax.contour(lons, lats, SFC_temp, np.arange(32, 33, 1), linewidths=1.5, colors='white', alpha=1, transform=datacrs, zorder=71)

    else:
        #Setting up the ranges for contoured temperatures
        clevs = np.arange(-42, 58, 3)

        #Creating a custom colortable using RGB values
        cmap_data = [(194/255, 190/255, 193/255),(235/255, 216/255, 230/255),(244/255,183/255,245/255),(237/255,137/255,239/255),(204/255,109/255,247/255),(213/255,0/255,255/255),(157/255,0/255,188/255),(106/255,0/255,156/255),(55/255,36/255,102/255),(44/255,81/255,216/255),(43/255,117/255,249/255),(44/255,158/255,252/255),(35/255,195/255,228/255),(24/255,219/255,197/255),(37/255,236/255,167/255),(74/255,248/255,128/255),(121/255,254/255,89/255),(164/255,252/255,60/255),(195/255,241/255,52/255),(223/255,223/255,55/255),(244/255,199/255,58/255),(253/255,172/255,52/255),(253/255,138/255,38/255),(243/255,99/255,21/255),(228/255,69/255,10/255),(206/255,45/255,4/255),(178/255,26/255,1/255),(142/255,10/255,1/255),(208/255,0/255,114/255),(255/255,0/255,142/255),(255/255,79/255,175/255),(255/255,125/255,196/255),(255/255,167/255,215/255)]
        cmap = colors.ListedColormap(cmap_data, 'temperature')
        norm = colors.BoundaryNorm(clevs, cmap.N)

        #Plotting 2m temperature with colorfill
        cf = ax.contourf(lons, lats, SFC_temp, np.arange(-42, 58, 3), cmap=cmo.tools.lighten(cmap, 0.6), transform=datacrs, zorder=65)

        #Creating a colorbar for 2m temperature
        cb = plt.colorbar(cf, orientation='horizontal', pad=0, aspect=50)
        cb.set_label('2m Temperature (°C)')

        #Plot a grey contour line at transition values for 2m temperatures
        cs22 = ax.contour(lons, lats, SFC_temp, np.arange(-42, 58, 3), linewidths=0.75, colors='grey', alpha=0.6, transform=datacrs, zorder=66)

        #Plot the freezing line
        cs28 = ax.contour(lons, lats, SFC_temp, np.arange(0, 1, 1), linewidths=1.5, colors='white', alpha=1, transform=datacrs, zorder=71)

    #Contouring 1000-500mb thickness values <5400m
    cs20 = ax.contour(lons, lats, Thickness, range(0, 5401, 60), transform=ccrs.PlateCarree(), colors='blue', linewidths=0.75, linestyles='dashed', zorder=75)
    cs20.clabel(fontsize=10, inline=1, inline_spacing=7, fmt='%i', rightside_up=True, use_clabeltext=True)

    #Contouring 1000-500mb thickness values >5400m
    cs27 = ax.contour(lons, lats, Thickness, range(5460, 10000, 60), transform=ccrs.PlateCarree(), colors='red', linewidths=0.75, linestyles='dashed', zorder=75)
    cs27.clabel(fontsize=10, inline=1, inline_spacing=7, fmt='%i', rightside_up=True, use_clabeltext=True)

    #Countoring MSLP
    cs3 = ax.contour(lons, lats, MSLP, range(900, 1050, 4), transform=ccrs.PlateCarree(), colors='k', linestyles='solid', zorder=77)
    cs3.clabel(fontsize=10, inline=1, inline_spacing=7, fmt='%i', rightside_up=True, use_clabeltext=True)

    #Add titles
    if TEMP == 'F':
        plt.title(modelName+' 2m Temp (shaded, °F), Freezing Line (white, 32°F), MSLP (black, hPa), and 1000-500mb Thickness (dashed, m)', loc='left')
    else:
        plt.title(modelName+' 2m Temp (shaded, °C), Freezing Line (white, 0°C), MSLP (black, hPa), and 1000-500mb Thickness (dashed, m)', loc='left')
    props = dict(boxstyle='round', facecolor='white')
    ax.text(0.828, 0.9825, validT, transform=ax.transAxes, fontsize=11.5, verticalalignment='top', bbox=props, zorder=100)

    #Save figure to your computer (save loction outlined near date selection)
    plt.savefig(fileNameSurface, dpi=200)

    #Clear figure variables for next figure creation
    plt.clf()

In [ ]:
#Handling map creation with if-then statements
print(Fore.BLUE + 'Map Creation Sequence Initiated')
print('-------------------------------')
if jet250 == 'Y':
    JetMap250()
    print('250mb Jet Map Created')
    print('-----')
else:
    print(Fore.RED + '250mb Jet Map **NOT** Created')
    print('-----')
    
if vorticity500 == 'Y':
    VorticityMap500()
    print('500mb Vorticity Map Created')
    print('-----')
else:
    print(Fore.RED + '500mb Vorticity Map **NOT** Created')
    print('-----')
    
if moisture850 == 'Y':
    MoistureMap850()
    print('850mb Moisture Map Created')
    print('-----')
else:
    print(Fore.RED + '850mb Moisture Map **NOT** Created')
    print('-----')

if frontogenesis925 == 'Y':
    FrontogenesisMap925()
    print('925mb Frontogenesis Map Created')
    print('-----')
else:
    print(Fore.RED + '925mb Frontogenesis Map **NOT** Created')
    print('-----')
    
if surface == 'Y':
    SurfaceMap()
    print('Surface Map Created')
else:
    print(Fore.RED + 'Surface Map **NOT** Created')